In [2]:
from __future__ import print_function
import os
import neat
import json

import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter
from explaneat.core.utility import one_hot_encode


from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, confusion_matrix


from copy import deepcopy

import time
from datetime import datetime


import gzip
try:
    import cPickle as pickle  # pylint: disable=import-error
except ImportError:
    import pickle  # pylint: disable=import-error

In [3]:

USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")


In [4]:
device

device(type='cpu')

# Iris Experiment

This experiment (a) test the experimental environment, but is also to evaluate the efficacy of the ExplaNEAT algorithm. Speed is a critical factor, as well as stability of results on population size. Total run time will also be measured

First, we need to set a random seed and a total stopping point in the number of generations

In [5]:
my_random_seed = 42
random.seed(my_random_seed)

In [6]:
def one_hot_encode(vals):
    width = max(vals)
    newVals = []
    for val in vals:
        blank = [0. for _ in range(width + 1)]
        blank[val] = 1.
        newVals.append(blank)
    return np.asarray(newVals)


## Dataset

We are going to work with the Iris dataset, which will be loaded from `sklearn`. We want to characterise the efficacy of the algorithm with regards to a mostly untransformed dataset, so we will only normalise the features

In [7]:
iris = datasets.load_iris()
xs_raw = iris.data[:, :]
scaler = StandardScaler()
scaler.fit(xs_raw)
xs = scaler.transform(xs_raw)
ys = iris.target
ys_onehot = one_hot_encode(ys)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(xs, ys, test_size=45, random_state=42)

In [9]:
# # xs = torch.from_numpy(xs).to(device)
# # ys = torch.from_numpy(ys).to(device)
# X_train = torch.from_numpy(X_train).to(device)
# X_test = torch.from_numpy(X_test).to(device)
# y_train = torch.from_numpy(y_train).to(device)
# y_test = torch.from_numpy(y_test).to(device)

Let's have a look at the data we are working with

In [10]:
xs[:5]

array([[-0.90068117,  1.01900435, -1.34022653, -1.3154443 ],
       [-1.14301691, -0.13197948, -1.34022653, -1.3154443 ],
       [-1.38535265,  0.32841405, -1.39706395, -1.3154443 ],
       [-1.50652052,  0.09821729, -1.2833891 , -1.3154443 ],
       [-1.02184904,  1.24920112, -1.34022653, -1.3154443 ]])

In [11]:
ys[:5]

array([0, 0, 0, 0, 0])

In [12]:
ys_onehot[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

## Performance metric

The NEAT implementation on which ExplaNEAT extends uses a single function call for evaluating fitness. Although this might be reworked for ExplaNEAT to be able to get consistency between the genome-evaluation and the backprop loss function, that can be reviewed later.

This use `CrossEntropyLoss` from `PyTorch`

In [13]:
def eval_genomes(genomes, config):
    loss = nn.CrossEntropyLoss()
    loss = loss.to(device)

    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in X_train:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(y_train)))

## The competition

In [20]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train);


In [38]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)
# Calculate the absolute errors
errors = abs(predictions - y_test)

In [39]:
print('Mean Absolute Error:', round(np.mean(errors), 2))

Mean Absolute Error: 0.01


In [40]:
predictions

array([1.001, 0.   , 1.995, 1.002, 1.12 , 0.   , 1.   , 1.969, 1.092,
       1.   , 1.955, 0.   , 0.   , 0.   , 0.   , 1.053, 2.   , 1.   ,
       1.001, 2.   , 0.   , 1.921, 0.   , 2.   , 2.   , 1.991, 1.986,
       2.   , 0.   , 0.   , 0.   , 0.   , 1.   , 0.   , 0.   , 1.96 ,
       1.003, 0.   , 0.   , 0.   , 1.983, 1.046, 1.002, 0.   , 0.   ])

In [41]:
errors = abs(predictions - y_test)

In [42]:
abs(predictions.round(0) - y_test)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [26]:
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')
# Accuracy: 93.99 %.

Accuracy: nan %.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [55]:
# Importing the classification report and confusion matrix
print(confusion_matrix(y_test,predictions.round(0)))

[[19  0  0]
 [ 0 13  0]
 [ 0  0 13]]


# SVM

In [45]:
from sklearn.svm import SVC
svm_model=SVC()


In [52]:
svm_model.fit(X_train, y_train)


/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [53]:
pred=svm_model.predict(X_test)


In [54]:
# Importing the classification report and confusion matrix
print(confusion_matrix(y_test,pred))

[[19  0  0]
 [ 0 13  0]
 [ 0  0 13]]
